In [55]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from datasets import load_dataset
import pandas as pd
import ast
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score




In [2]:
#dowlnoad the dataset
data_load = load_dataset("nedjmaou/MLMA_hate_speech")
data_train = data_load["train"]
dataset = data_train.to_pandas()
print(dataset.head())
print(dataset.shape)

   HITId                                              tweet       sentiment  \
0      1  صلاة الفجر خير لك من ترديد بول البعير وسبي الن...  hateful_normal   
1      2  صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...       offensive   
2      3  طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...       offensive   
3      4  @user @user انا اوافقك بخصوص السوريين و العراق...          normal   
4      5  هذه السعودية التي شعبها شعب الخيم و بول البعير...          normal   

  directness                           annotator_sentiment  target       group  
0   indirect                                         shock  gender  individual  
1   indirect  anger_confusion_sadness_indifference_disgust   other       other  
2   indirect                                  indifference   other  individual  
3     direct                                  indifference  origin       other  
4   indirect                                  indifference  origin       other  
(18661, 7)


In [3]:
#dataset separate by languages
arab_path = "/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/ar_dataset.csv"
ar_dataset = pd.read_csv(arab_path)
print(ar_dataset.shape)
print(ar_dataset.head())


en_path="/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/en_dataset.csv"
en_dataset = pd.read_csv(en_path)
print(en_dataset.shape)
print(en_dataset.head())

fr_path="/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/fr_dataset.csv"
fr_dataset = pd.read_csv(fr_path)
print(fr_dataset.shape)
print(fr_dataset.head())


(3353, 7)
   HITId                                              tweet       sentiment  \
0      1  صلاة الفجر خير لك من ترديد بول البعير وسبي الن...  hateful_normal   
1      2  صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...       offensive   
2      3  طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...       offensive   
3      4  @user @user انا اوافقك بخصوص السوريين و العراق...          normal   
4      5  هذه السعودية التي شعبها شعب الخيم و بول البعير...          normal   

  directness                           annotator_sentiment  target       group  
0   indirect                                         shock  gender  individual  
1   indirect  anger_confusion_sadness_indifference_disgust   other       other  
2   indirect                                  indifference   other  individual  
3     direct                                  indifference  origin       other  
4   indirect                                  indifference  origin       other  
(5647, 7)
   HITId           

In [3]:
#get binary data (ground truth)
dataset['sentiment'].unique()
dataset['sentiment'] = dataset['sentiment'].apply(lambda x: 0 if x=='normal'else 1)
dataset['sentiment'].value_counts()



sentiment
1    15603
0     3058
Name: count, dtype: int64

In [50]:
#One by One

used_data = en_dataset.iloc[0:40]
predictions_concat=[]

# Wrap range with tqdm
for i in tqdm(range(used_data.shape[0])):
    tweet = en_dataset['tweet'].iloc[i]
    # prompt
    prompt = [{"role": "system", "content": "Do you think these tweets are hate speech, offensive or normal speech. Try your best even if we do not have context. Display the reasult as an integer of 0 (Normal), 1 (offensive) and 2 (hate). Do not display anything else, it is important that only the interger is displayed."},
              {"role": "user", "content": tweet},]
    # LLM initialization
    llm = Ollama(model="llama3")
    predictions = llm.invoke(prompt)
    predictions = int(predictions) 
    predictions_concat.append(predictions)

print(len(predictions_concat))
#add the predictions to the dataset
used_data['predictions'] = predictions_concat


  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:30<00:00,  1.33it/s]
/var/folders/gm/941r48zd5qxcl4tb3v9q30440000gn/T/ipykernel_93745/2836036701.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_data['predictions'] = predictions_concat


40


,HITId,tweet,sentiment,directness,annotator_sentiment,target,group,predictions
0,1,call sis im boy girl still faggot shut,offensive,indirect,confusion_disgust,origin,gay,2
1,2,@user @user legit nilas retarded idk,offensive_disrespectful,indirect,anger_confusion,disability,special_needs,2
2,3,said retard @url,fearful_abusive_offensive_hateful,indirect,confusion_disgust,disability,special_needs,2
3,4,america another 8 years obama's ideology via h...,fearful_abusive_hateful_disrespectful_normal,indirect,anger_fear_shock_sadness_disgust,origin,other,2
4,5,@user don... fucking made cry twat.,offensive,indirect,shock_disgust,gender,women,2
5,6,one wants 2 negro swan,offensive,indirect,anger_sadness,origin,left_wing_people,2
6,7,@user @user @user hahaha. one trump's shithole...,offensive,indirect,confusion_sadness_indifference_disgust,origin,other,2
7,8,u juul ur faggot,offensive,indirect,confusion_fear_indifference_sadness_disgust,sexual_orientation,other,2
8,9,@user @user even called country shithole peopl...,offensive,indirect,anger_fear_indifference_sadness_disgust,origin,special_needs,2
9,10,canadians never met seen associated person cur...,offensive,indirect,sadness_indifference,disability,special_needs,2


In [56]:
#Metrics
acc=accuracy_score(used_data['sentiment'], used_data['predictions'])
F1=f1_score(used_data['sentiment'], used_data['predictions'], average='macro')
Precision=precision_score(used_data['sentiment'], used_data['predictions'], average='macro')
Recall=recall_score(used_data['sentiment'], used_data['predictions'], average='macro')

ValueError: Mix of label input types (string and number)